In [0]:
from pyspark.sql.functions import *

from src.data.data_utils import write_delta_table

In [0]:
catalog_dev = "`land_auto-gen-kart_dev`"
schema_dev = "dl_bildesegmentering"
spark.sql(f"USE CATALOG {catalog_dev}")
spark.sql(f"CREATE SCHEMA IF NOT EXISTS {schema_dev}")
spark.sql(f"USE SCHEMA {schema_dev}")
log_table = "logs_predicted_helipads"
silver_table = "predicted_helipads_silver"
gold_table = "predicted_helipads_gold"

In [0]:
q = f"""
CREATE TABLE IF NOT EXISTS {gold_table} (
    row_hash STRING,
    geometry BINARY,
    source_file STRING,
    ingest_time TIMESTAMP,
    photo_time DATE
) USING DELTA
"""
spark.sql(q)

In [0]:
df_silver = spark.read.table(silver_table)

df = (
    df_silver.withColumn("geometry", expr("ST_Point(centroid_x, centroid_y)"))
    .withColumn("geometry", expr("ST_AsBinary(geometry)"))
    .drop("centroid_x", "centroid_y")
)  # Lager en punktgeometri fra koordinatene

In [0]:
write_delta_table(df, gold_table, id_col="row_hash")